# Serving Demo — FastAPI + Transformers pipeline
**TL;DR:** Expose a sentiment pipeline via FastAPI, test with an in-notebook client, and benchmark the core inference.

**Models & Datasets:** [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) (Apache-2.0), [IMDB samples](https://huggingface.co/datasets/imdb) (CC BY-NC 4.0)
**Run Profiles:** 🖥️ CPU | 🍎 Metal (Apple Silicon) | 🧪 Colab/T4 | ⚡ CUDA GPU
**Env (minimal):** python>=3.10, transformers, datasets, evaluate, accelerate (optional: peft, bitsandbytes, timm, diffusers)
**Colab:** [Open in Colab](https://colab.research.google.com/github/SSusantAchary/Hands-On-Huggingface-AI-Models/blob/main/notebooks/serving/serving-fastapi-pipeline_demo_cpu-first.ipynb)

**Switches (edit in one place):**
- `device` = {"cpu","mps","cuda"}
- `precision` = {"fp32","fp16","bf16","int8","4bit"}  (apply only if supported)
- `context_len` / `image_res` / `batch_size`

**Footprint & Speed (fill after run):**
- Peak RAM: TODO
- Peak VRAM: TODO (if GPU)
- TTFB: TODO, Throughput: TODO, Load time: TODO

**Gotchas:** Use FastAPI TestClient inside notebooks to avoid event loop clashes ([Fixes & Tips](../fixes-and-tips/fastapi-local-run.md))



## Setup
Load the sentiment pipeline and initialise a FastAPI app.


In [ ]:

import json
import os
import subprocess
import time
from pathlib import Path
from typing import Dict

import torch
from datasets import load_dataset
from transformers import pipeline

from notebooks._templates.measure import append_benchmark_row, measure_memory_speed

try:
    from fastapi import FastAPI
    from pydantic import BaseModel
    from fastapi.testclient import TestClient
except ImportError as exc:  # noqa: BLE001
    raise RuntimeError("Install fastapi and pydantic to run this notebook") from exc

DEVICE_PREFERENCE = os.environ.get("HF_DEVICE", "cpu")
PRECISION = os.environ.get("HF_PRECISION", "fp32")

def resolve_device(preference: str = "cpu") -> str:
    if preference == "cuda" and torch.cuda.is_available():
        return "cuda:0"
    if preference == "mps" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = resolve_device(DEVICE_PREFERENCE)
print(f"Using device={DEVICE}")

MODEL_ID = "distilbert-base-uncased-finetuned-sst-2-english"
OUTPUT_DIR = Path("outputs") / "fastapi-serving"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

sample_texts = load_dataset("imdb", split="test[:4]")["text"]


## Build the API


In [ ]:

torch.manual_seed(42)

load_start = time.perf_counter()
classifier = pipeline(
    "text-classification",
    model=MODEL_ID,
    device=DEVICE,
    top_k=1,
)
load_time = time.perf_counter() - load_start

app = FastAPI(title="Sentiment Demo")

class Request(BaseModel):
    text: str

@app.post("/predict")
def predict(request: Request) -> Dict[str, str]:
    pred = classifier(request.text)[0]
    return {"label": pred["label"], "score": f"{pred['score']:.4f}"}

client = TestClient(app)

payload = {"text": "I love CPU-friendly ML demos."}
response = client.post("/predict", json=payload)
response.json()


## Measurement


In [ ]:

def run_inference(recorder):
    for idx, text in enumerate(sample_texts):
        _ = classifier(text)
        if idx == 0:
            recorder.mark_first_token()
        recorder.add_items(1)

metrics = measure_memory_speed(run_inference)

def fmt(value, digits=4):
    if value in (None, "", float("inf")):
        return ""
    return f"{value:.{digits}f}"

try:
    repo_commit = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:  # noqa: BLE001
    repo_commit = ""

append_benchmark_row(
    task="fastapi-serving",
    model_id=MODEL_ID,
    dataset="imdb",
    sequence_or_image_res="256-tokens",
    batch="1",
    peak_ram_mb=fmt(metrics.get("peak_ram_mb"), 2),
    peak_vram_mb=fmt(metrics.get("peak_vram_mb"), 2),
    load_time_s=fmt(load_time, 2),
    ttfb_s=fmt(metrics.get("ttfb_s"), 3),
    tokens_per_s_or_images_per_s=fmt(metrics.get("throughput_per_s"), 3),
    precision=PRECISION,
    notebook_path="notebooks/serving/serving-fastapi-pipeline_demo_cpu-first.ipynb",
    repo_commit=repo_commit,
)

with open(OUTPUT_DIR / "metrics.json", "w", encoding="utf-8") as fp:
    json.dump(metrics, fp, indent=2)
metrics


## Results Summary
        - Observations: TODO
        - Metrics captured: see `benchmarks/matrix.csv`

        ## Next Steps
        - TODOs: fill in after benchmarking

        ## Repro
        - Seed: 42 (set in measurement cell)
        - Libraries: captured via `detect_env()`
        - Notebook path: `notebooks/serving/serving-fastapi-pipeline_demo_cpu-first.ipynb`
        - Latest commit: populated automatically when appending benchmarks (if git available)
